In [9]:
import os
import sys
from quetzal.os.parallel_call import parallel_call_notebook, parallel_call_notebooks

In [10]:
if not os.path.exists('log/'):
    os.makedirs('log/')

In [59]:
input_path = '../../input/'

# Launcher

This notebook automatically launches all operations for a complete model run. One can decide to skip network preparation steps, as they take multiple hours for the region of Germany and the networks are readily included in the repository.

Detailed explainations of certain steps can be found in the corresponding notebook. All ASSUMPTIONS are gathered in the `parameters.xls` file. This file also includes scenarios.

In [49]:
kwargs = {'workers':2, 'errout_suffix':True, 'sleep':1,'stdout_path':r'log/out.txt', 'stderr_path':r'log/err.txt'}

In [35]:
param_1 = [1,2,3]
param_2 = [50,60]
param_3 = [True, False]


In [61]:
param_combinations = []
n=0
for p_1 in param_1:
    for p_2 in param_2:
        for p_3 in param_3:
            param_combinations.append((n,(p_1, p_2, p_3)))
            n+=1


param_combinations
# save them to file
param_combinations_dict = dict((str(n),x) for n,x in param_combinations)
with open(input_path+'param_combinations_dict.txt', 'w') as f:
    print(param_combinations_dict, file=f)



In [79]:
# for each notebook read in param file based on scenario number
# configure parameters in notebooks based on file
import ast
with open(input_path+'param_combinations_dict.txt', "r") as file:
    param_1, param_2, param_3 = ast.literal_eval(file.read())["4"]
# file.close()


In [47]:
scenarios = [str(n) for n, x in param_combinations]

In [13]:
import time
start = time.time()

In [81]:
# parallel_call_notebook('001_test_environment.ipynb', arg_list=scenarios, **kwargs)


# Zone preparation


In [ ]:
# Zone creation
parallel_call_notebook('101_p_create_zones.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('102_OMS_POIs.ipynb', arg_list=scenarios, **kwargs)

# Network preparation


In [ ]:
# Access/egress distances to PT services based on networks and census data
parallel_call_notebook('Exercise1_network_creation.ipynb', arg_list=scenarios, **kwargs)


In [ ]:
# for scenario in scenarios:
#     parallel_call_notebook('Exercise2_OD-LoS-stack.ipynb', arg_list=[scenario], **kwargs)
#     parallel_call_notebook('Exercise3_Four_steps.ipynb', arg_list=[scenario], **kwargs)

In [ ]:
parallel_call_notebook('Exercise2_OD-LoS-stack.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('Exercise3_Four_steps.ipynb', arg_list=scenarios, **kwargs)

# Pathfinders and level of service

# Estimation

# Modelling

Except for the logit model step, all modelling steps require access to the OD volumes dataset.

In [ ]:
# Total execution time
end = time.time()
int(end - start)